In [1]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

import matplotlib.pyplot as plt
import matplotlib.animation as animation
import bar_chart_race as bcr
import cv2
import numpy as np

ModuleNotFoundError: No module named 'bar_chart_race'

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FusionModel(nn.Module):
    def __init__(self):
        super(FusionModel, self).__init__()
        self.relu=nn.ReLU(inplace=True)
        
        ## Hint: Please refer to above table for constructing layers
        # Construct block of RGB layers which takes RGB channel(3) as input

        #______________________________________________________________________________________________________
        # RGB layers
        self.rgb_conv1 = nn.Conv3d(3, 64, kernel_size=3, stride=1, padding=1)
        self.rgb_bn1 = nn.BatchNorm3d(64)
        #______________________________________________________________________________________________________

        # Construct block of optical flow layers which takes the optical flow channel(2) as input
        #______________________________________________________________________________________________________
        # Optical flow layers
        self.opt_conv1 = nn.Conv3d(2, 64, kernel_size=3, stride=1, padding=1)
        self.opt_bn1 = nn.BatchNorm3d(64)
        #______________________________________________________________________________________________________

        # Construct merging Block
        
        # Fully Connected Layers
        self.fc1 = nn.Linear(128, 128)
        self.dropout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(128, 32)
        self.fc3 = nn.Linear(32, 2)
        
        # Initialize weights
        self.__init_weight()
        
    def forward(self, x):
        rgb = x[...,:3] # Seperate RGB data
        opt = x[...,3:5] # Seperate Optical flow data
        # Reshpae tensor into (#Batch, channel, dimension(#frame), height, width)
        rgb = rgb.contiguous().view(-1, 3, 64, rgb.shape[2], rgb.shape[3]) # 64 is fixed number of frames
        opt = opt.contiguous().view(-1, 2, 64, opt.shape[2], opt.shape[3]) 

        # Pass through the RGB data through the blocks of RGB layers
        
        # Pass through the optical flow data through the blocks of RGB layers
        
        # Fuse by performing elementwise multiplication of rgb and opt tensors. 
        # fused = ## fill here ##
        #______________________________________________________________________________________________________
        fused = rgb * opt
        # # Perform maxpooling of fused
        # fused = ## fill here ##
        fused = F.max_pool3d(fused, kernel_size=2, stride=2)
        #______________________________________________________________________________________________________

        # Pass through the fused data into merging block 

        #______________________________________________________________________________________________________
        # Merging block
        merged = torch.cat((rgb, opt), dim=1)
        # Flatten or further process merged as required
        merged = merged.view(merged.size(0), -1)
        #______________________________________________________________________________________________________


        
        # Fully Connected Layers
        x = merged.view(merged.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        
        return x
    
    def __init_weight(self):
        for m in self.modules():
            if isinstance(m, nn.Conv3d):
                # Perform weight initialization ("kaiming normal")
                #______________________________________________________________________________________________________
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
                #______________________________________________________________________________________________________


SyntaxError: incomplete input (2113503041.py, line 59)

In [ ]:
device = torch.device('cuda')

model = FusionModel().to(device)
# user your pretrained model path
model.load_state_dict(torch.load('./FusionModel.pth'))

<All keys matched successfully>

In [ ]:
def uniform_sampling(video, target_frames=64):
    # get total frames of input video and calculate sampling interval
    len_frames = int(len(video))
    interval = int(np.ceil(len_frames/target_frames))
    # init empty list for sampled video and
    sampled_video = []
    for i in range(0,len_frames,interval):
        sampled_video.append(video[i])
    # calculate numer of padded frames and fix it
    num_pad = target_frames - len(sampled_video)
    padding = []
    if num_pad>0:
        for i in range(-num_pad,0):
            try:
                padding.append(video[i])
            except:
                padding.append(video[0])
        sampled_video += padding
    # get sampled video
    return np.array(sampled_video, dtype=np.float32)


def normalize(data):
    mean = data.mean()
    std = data.std()
    return (data - mean) / std

In [ ]:
# test video path
file_path ="../data/visualization/Assault018_x264.mp4"

cap = cv2.VideoCapture(file_path)
# Get number of frames
len_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

frames = []
flows = []
preds = []
resize = (224, 224)

prev_frame = None
for iter in range(0, len_frames-1):
    _, frame = cap.read()
    frame = cv2.resize(frame, resize, interpolation=cv2.INTER_AREA)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = np.reshape(frame, (224, 224, 3))
    frames.append(frame)

    img = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    gray_img = np.reshape(img, (224, 224, 1))
    if prev_frame is None:
        prev_frame = gray_img

    flow = cv2.calcOpticalFlowFarneback(prev_frame, gray_img, None, 0.5, 3, 15, 3, 5, 1.2,
                                        cv2.OPTFLOW_FARNEBACK_GAUSSIAN)
    prev_frame = gray_img
    # subtract the mean in order to eliminate the movement of camera
    flow[..., 0] -= np.mean(flow[..., 0])
    flow[..., 1] -= np.mean(flow[..., 1])
    # normalize each component in optical flow
    flow[..., 0] = cv2.normalize(flow[..., 0], None, 0, 255, cv2.NORM_MINMAX)
    flow[..., 1] = cv2.normalize(flow[..., 1], None, 0, 255, cv2.NORM_MINMAX)

    flows.append(flow)
    result = np.zeros((len(flows), 224, 224, 5))
    result[..., :3] = frames
    result[..., 3:] = flows

    data = np.float32(result)
    # # sampling 64 frames uniformly from the entire video
    data = uniform_sampling(video=data, target_frames=64)
    # normalize rgb images and optical flows, respectively
    data[..., :3] = normalize(data[..., :3])
    data[..., 3:] = normalize(data[..., 3:])

    fr, w, h, ch = data.shape
    data = data.reshape((-1, fr, w, h, ch))
    # pred = model.predict(data)[0]
    pred = model(torch.Tensor(data).to(device))
    # fights.append(pred[0])
    # none.append(pred[1])
    preds.append(pred[0].detach().cpu().numpy())

cap.release()

In [ ]:
import pandas as pd

print(np.array(preds).shape)

group_list = ["Violence", "Non-Violence"]
df = pd.DataFrame(preds, columns = ['Violence', 'Non-Violence'])

df

(360, 2)


,Violence,Non-Violence
0,-0.318526,0.229631
1,-0.223172,0.176996
2,-0.262685,0.200812
3,-0.251993,0.101380
4,-0.561792,0.476182
...,...,...
355,0.009180,-0.093508
356,-0.096298,0.025228
357,-0.025634,-0.138630
358,0.035569,-0.063577


In [ ]:
bcr.bar_chart_race(df=df[:],
                   n_bars = 2,
                   figsize=(4, 4),
                   label_bars=False,
                   sort='desc',
                   #title='Violence detection',
                   fixed_order=['Violence', 'Non-Violence'],
                   orientation='h',
                   fixed_max=True,
                   period_length=int(1000/fps),
                  )